In [1]:
using ProgressMeter
using JLD
using Random
include("../alpcahus.jl")
include("../helperFunctions.jl")
include("../cluster.jl")
include("../hemppcatFunctions.jl")
include("../otherMethods.jl")

correlation_kernel_sparse (generic function with 1 method)

In [2]:
X = randn(50,250)
D = size(X)[1]
d = 5
K = 2
#p = [10;240]
variances = 10*rand(250);

In [3]:
c = tscWrapper(X, 10; K=K)
T1 = svd(X[:,c .== 1])
T2 = svd(X[:,c .== 2])
F1 = T1.U[:,1:d]*Diagonal(sqrt.(T1.S[1:d]))
F2 = T2.U[:,1:d]*Diagonal(sqrt.(T2.S[1:d]))

#F1 = svd(randn(D,d)).U
#F2 = svd(randn(D,d)).U
F_init = [F1 , F2] # Factor matrices
μ_init = zeros(D, K) # Mixture means
π_init = [0.5, 0.5] # Mixing proportions
#weighted_var = (p[1]/(p[2]+p[1]))*variances[1] + (p[2]/(p[2]+p[1]))*variances[2] # smooth variance based on points for MPPCA
v_init = [1;1]#variances#[weighted_var, weighted_var] # Noise variances
n = 250#[2*p[1], 2*p[2]] # number of points in each variance group
# Run MPPCA
model = create_MPPCAModel(F_init, μ_init, π_init, v_init, sum(n)) 
#results, _ = mppca(model, X; eps=1e-6); 

MPPCAModel([[0.5629077772917 -0.28630604802116993 … -0.6265230363396527 0.07367523858468893; -0.7215472105675144 -0.4602273180843877 … 0.06691307948407191 0.8743273202178653; … ; 0.7464699023994982 0.3532951500149899 … -0.7626984828505156 0.8363490113521066; 0.6391082790821686 -0.4797603721254946 … -0.5337419624766004 -0.1642520501032289], [0.18644743478837364 -1.5169107166141986 … -0.7678341297970659 0.01108680239401094; 0.0862466705496027 -0.3153980320199336 … 0.8740044867800288 0.3798998995020047; … ; 0.6077764231850343 0.9766742501904891 … -0.32531901739960994 -0.34263912875245756; 0.18002426502211635 -0.11840400561053556 … 0.6676160943689787 -1.0658170845351627]], [0.0 0.0; 0.0 0.0; … ; 0.0 0.0; 0.0 0.0], [0.5, 0.5], [1, 1], 250, [[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]], [[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0

In [28]:
function MPPCA_Wrapper(X::Matrix, K::Int, q::Int, d::Int)
    # TWO CLUSTERS ONLY
    D,N = size(X)
    
    c = tscWrapper(X, q; K=K)
    T1 = svd(X[:,c .== 1])
    T2 = svd(X[:,c .== 2])
    F1 = T1.U[:,1:d]*Diagonal(sqrt.(T1.S[1:d]))
    F2 = T2.U[:,1:d]*Diagonal(sqrt.(T2.S[1:d]))

    F_init = [F1 , F2] # Factor matrices
    μ_init = zeros(D, K) # Mixture means
    π_init = [0.5, 0.5] # Mixing proportions
    v_init = [1.0; 1.0]

    model = create_MPPCAModel(F_init, μ_init, π_init, v_init, N) 
    results, _ = mppca(model, X; eps=1e-4); 

    U1 = svd(results.F[1]).U
    U2 = svd(results.F[2]).U

    labelsMPPCA = clusterAssignment(X, [U1,U2], K)
    
    return labelsMPPCA
end

MPPCA_Wrapper (generic function with 1 method)

In [29]:
MPPCA_Wrapper(X,2,10,5)

250-element Vector{Int64}:
 2
 2
 2
 2
 2
 1
 1
 2
 2
 2
 2
 2
 2
 ⋮
 2
 1
 1
 1
 1
 2
 1
 2
 2
 2
 1
 2